In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import tensorflow as tf
from tqdm import tqdm
from glob import glob
from google.colab import drive

import librosa
import librosa.display as dsp
import IPython.display as ipd

warnings.filterwarnings(action='ignore')
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/deep-voice/data/'

Mounted at /content/drive
/content/drive/MyDrive/deep-voice/data


In [2]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당

In [3]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(813)

In [4]:
# 저장된 데이터 불러오기
X_train_mel = np.load('X_train_mel.npy')
X_val_mel = np.load('X_val_mel.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, optimizers, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dropout, Dense, Lambda, Reshape
from tensorflow.keras.applications.vgg19 import preprocess_input

# 입력 차원을 (128, 2970, 1)에서 (128, 128, 3)으로 변경하는 사용자 정의 함수
def preprocess_image(x):
    # 1채널 grayscale 이미지를 3채널 RGB 이미지로 변환
    x = tf.image.grayscale_to_rgb(x)
    # 이미지 크기를 (128, 128)으로 조정
    x = tf.image.resize(x, [128, 128])
    return x

input_shape = (128, 2970, 1)

vgg19 = tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(128, 128, 3)  # input_shape를 (128, 128, 3)으로 변경
)

# VGG19의 모든 계층을 동결합니다.
for layer in vgg19.layers:
    layer.trainable = False

# Sequential 모델을 구성합니다.
model = Sequential([
    Lambda(preprocess_image, input_shape=input_shape),  # 입력 데이터 차원 및 채널 변경 및 크기 조정
    vgg19,
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')  # 최종 출력 계층입니다. 분류하고자 하는 클래스 수에 맞게 조정하세요.
])

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 128, 128, 3)       0         
                                                                 
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dropout (Dropout)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 512)               4194816   
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 24,220,226
Trainable params: 4,195,842
Non

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(
    monitor='val_loss',  # 모니터링할 데이터
    patience=10,         # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # Early stopping의 진행 상황을 어떻게 표시할 것인지
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원할지 여부
)

# 모델 훈련
history = model.fit(
    X_train_mel, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mel, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 425s 11s/step - loss: 0.3121 - accuracy: 0.9742 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
40/40 [==============================] - 481s 12s/step - loss: 0.0146 - accuracy: 0.9992 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
40/40 [==============================] - 431s 11s/step - loss: 0.0332 - accuracy: 0.9992 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
40/40 [==============================] - 426s 11s/step - loss: 1.5775e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
40/40 [==============================] - 481s 12s/step - loss: 1.0910e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
38/40 [===========================>..] - ETA: 16s - loss: 3.9350e-05 - accuracy: 1.0000